# Select Model

In [ ]:
from __future__ import annotations

TARGET_MODEL = "distilbert/distilbert-base-uncased"
#TARGET_MODEL = "microsoft/deberta-v3-large"
#TARGET_MODEL = "microsoft/Phi-3-mini-128k-instruct"

In [ ]:
# Directory settings

from pathlib import Path

OUTPUT_DIR = Path("/mnt/batch/tasks/shared/LS_root/mounts/clusters/cn1/code/Users/CN/Output/")
#OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

INPUT_DIR = Path("/mnt/batch/tasks/shared/LS_root/mounts/clusters/cn1/code/Users/CN/Input_preprocess/")

# Import data and basic EDA

In [ ]:
# import data with input and target
import pandas as pd
import json
train = pd.read_csv(str(INPUT_DIR)+'/preprocessed.csv')

In [ ]:
print(train.shape)
display(train.head())
print(train.scored.value_counts())

# Preproccessing

In [ ]:
train['label'] = train['scored']

In [ ]:
train = train[['label', 'text_parsed']]

In [ ]:
train.head()

# Modeling

In [ ]:
# create folds

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = train.loc[:, train.columns != "label"]
y = train.loc[:, train.columns == "label"]

for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    train.loc[valid_index, "fold"] = i
    
print(train.groupby("fold")["label"].value_counts())
train.head()

In [ ]:
# establish validation data

val = train[train["fold"] == 0]
train = train[train["fold"] != 0]

In [ ]:
train_df = train[["text_parsed", "label"]]
valid_df = val[["text_parsed", "label"]]

print(train_df.shape)
print(train_df.label.value_counts())
print(valid_df.shape)
print(valid_df.label.value_counts())

In [ ]:
# import datasets

#!pip install -q -U peft --no-index --find-links file:///mnt/batch/tasks/shared/LS_root/mounts/clusters/cn1/code/Users/CN/Input/
#!pip install -q -U accelerate --no-index --find-links file:///mnt/batch/tasks/shared/LS_root/mounts/clusters/cn1/code/Users/CN/Input/
#!pip install -q -U bitsandbytes --no-index --find-links file:///mnt/batch/tasks/shared/LS_root/mounts/clusters/cn1/code/Users/CN/Input/
#!pip install -q -U transformers --no-index --find-links file:///mnt/batch/tasks/shared/LS_root/mounts/clusters/cn1/code/Users/CN/Input/
#!pip install torch

%pip install -q -U peft
%pip install -q -U accelerate
%pip install -q -U bitsandbytes
%pip install -q -U transformers
%pip install -q -U sentencepiece 
%pip install torch
%pip install datasets

In [ ]:
# load model with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

quantized = False
if quantized:

    peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        target_modules=[
            "q_proj",
            "v_proj"
        ],
    )

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    #bnb_config = BitsAndBytesConfig(
    #    load_in_8bit=True,
    #    bnb_8bit_quant_type="nf8",
    #    bnb_8bit_use_double_quant=True,
    #    bnb_8bit_compute_dtype=torch.float16
    #)

In [ ]:
from transformers import AutoTokenizer, LlamaForSequenceClassification, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
if quantized:
    base_model = LlamaForSequenceClassification.from_pretrained(
        TARGET_MODEL,
        num_labels=5,
        quantization_config=bnb_config,   #TO QUANTIZE
        device_map={"":0}
    )
    base_model.config.pretraining_tp = 1 # 1 is 7b
    base_model.config.pad_token_id = tokenizer.pad_token_id

if not quantized:
    base_model = AutoModelForSequenceClassification.from_pretrained(
        TARGET_MODEL,
        num_labels=5,
        device_map={"":0}
    )
    base_model.config.pretraining_tp = 1 # 1 is 7b
    base_model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
if quantized:
    model = get_peft_model(base_model, peft_config)

if not quantized:   
    model = base_model

In [ ]:
if quantized:
    model.print_trainable_parameters()

In [ ]:
print(train_df.shape)
print(valid_df.shape)

In [ ]:
print(train_df.label.value_counts(), valid_df.label.value_counts())

In [ ]:
# datasets
from datasets import Dataset

# from pandas
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)

In [ ]:
def preprocess_function(examples, max_length=1024):   
    return tokenizer(examples["text_parsed"], truncation=True, max_length=max_length, padding=True)

In [ ]:
train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
valid_tokenized_ds = valid_ds.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(np.array(predictions), axis=1)

    unique_classes = np.unique(labels)
    class_mae_values = []
    
    for cls in unique_classes:
        cls_indices = np.where(labels == cls)
        cls_labels = labels[cls_indices]
        cls_predictions = predictions[cls_indices]
        
        if len(cls_labels) > 0:  # Avoid division by zero
            cls_mae = mean_absolute_error(cls_labels, cls_predictions)
            class_mae_values.append(cls_mae)
    
    mean_class_mae = np.mean(class_mae_values) if class_mae_values else None
    
    return {
        "mean_class_mae": mean_class_mae,
    }


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

steps = 25

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    max_grad_norm=0.3,
    optim='paged_adamw_32bit',
    lr_scheduler_type="cosine",
    num_train_epochs=10,
    weight_decay=1e-5,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=False,
    warmup_ratio=0.1,
    eval_steps=steps,
    logging_steps=steps,
    report_to='none'
)

early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_ds,
    eval_dataset=valid_tokenized_ds,
    tokenizer=tokenizer,
    callbacks=[early_stopping],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
from shutil import rmtree

trainer.save_model(output_dir=str(OUTPUT_DIR))

for path in Path(training_args.output_dir).glob("checkpoint-*"):
    if path.is_dir():
        rmtree(path)

In [ ]:
del trainer, model, base_model

In [ ]:
# cuda cache clear
import torch
torch.cuda.empty_cache()

# Test loading model

In [ ]:
# load model / tokenizer with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

quantized = False
if quantized:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

In [ ]:
from transformers import AutoTokenizer, LlamaForSequenceClassification, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
if quantized:
    base_model = LlamaForSequenceClassification.from_pretrained(
        TARGET_MODEL,
        num_labels=5,
        quantization_config=bnb_config,   #TO QUANTIZE
        device_map={"":0}
    )
    base_model.config.pretraining_tp = 1 # 1 is 7b
    base_model.config.pad_token_id = tokenizer.pad_token_id

if not quantized:
    base_model = AutoModelForSequenceClassification.from_pretrained(
        TARGET_MODEL,
        num_labels=5,
        device_map={"":0}
    )
    base_model.config.pretraining_tp = 1 # 1 is 7b
    base_model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
if quantized:
    model = get_peft_model(base_model, peft_config)

if not quantized:   
    model = base_model

In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
pred_output = trainer.predict(valid_tokenized_ds)
logits = pred_output.predictions
logits

In [ ]:
probs = np.argmax(logits, axis=1)

probs

In [ ]:
sub = valid_df.copy()
sub['predictions'] = probs
# sub.to_csv('submission.csv', index=False)
sub.head(50)

# Test for promptflow

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
%pip install -q -U peft
%pip install -q -U accelerate
%pip install -q -U bitsandbytes
%pip install -q -U transformers
%pip install -q -U sentencepiece 
%pip install torch
%pip install datasets
from datasets import Dataset

TARGET_MODEL = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/cn1/code/Users/CN/Output/"
INPUT_DIR = Path("/mnt/batch/tasks/shared/LS_root/mounts/clusters/cn1/code/Users/CN/Input_preprocess/")
train = pd.read_csv(str(INPUT_DIR)+'/preprocessed.csv')
train['label'] = train['scored']
train = train[['label', 'text_parsed']]
train_df = train[["text_parsed", "label"]]
train_ds = Dataset.from_pandas(train_df)

# load model / tokenizer with 4bit bnb
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

quantized = False
if quantized:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

from transformers import AutoTokenizer, LlamaForSequenceClassification, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def preprocess_function(examples, max_length=1024):   
    return tokenizer(examples["text_parsed"], truncation=True, max_length=max_length, padding=True)
train_tokenized_ds = train_ds.map(preprocess_function, batched=True)

from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

if quantized:
    base_model = LlamaForSequenceClassification.from_pretrained(
        TARGET_MODEL,
        num_labels=5,
        quantization_config=bnb_config,   #TO QUANTIZE
        device_map={"":0}
    )
    base_model.config.pretraining_tp = 1 # 1 is 7b
    base_model.config.pad_token_id = tokenizer.pad_token_id

if not quantized:
    base_model = AutoModelForSequenceClassification.from_pretrained(
        TARGET_MODEL,
        num_labels=5,
        device_map={"":0}
    )
    base_model.config.pretraining_tp = 1 # 1 is 7b
    base_model.config.pad_token_id = tokenizer.pad_token_id

if quantized:
    model = get_peft_model(base_model, peft_config)

if not quantized:   
    model = base_model

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

pred_output = trainer.predict(train_tokenized_ds)
logits = pred_output.predictions

probs = np.argmax(logits, axis=1)

sub = train_df.copy()
sub['predictions'] = probs
sub.head(50)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 593/593 [00:00<00:00, 1514.09 examples/s]
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


,text_parsed,label,predictions
0,"Starting OK, go for it.",4,1
1,"Alright, thank you.",0,0
2,Thank you.,0,0
3,So this general path thing just allows us to m...,4,4
4,Why is that important?,4,4
5,Context is key.,0,0
6,"When using any type of generative AI, it's onl...",1,1
7,"So in this context, what we wanted to understa...",3,2
8,I took these based off of the objectives you p...,1,1
9,Nicole and Jerry.,0,0
